In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../')  # Add the parent folder to the Python path

from sql_functions import get_dataframe  # Import the function

main_cities = [
    "New York, NY",
    "Boston, MA",
    "Washington, DC",
    "Newark, NJ",
]

side_cities = [
    "Miami, FL",
    "Houston, TX",
    "San Francisco, CA",
    "Seattle, WA"
]


In [4]:
flights_2015 = get_dataframe('SELECT * FROM group3.flights_2015')
flights_2016 = get_dataframe('SELECT * FROM group3.flights_2016')
flights_2017 = get_dataframe('SELECT * FROM group3.flights_2017')

In [ ]:
flights_2016[flights_2016[main]]